In [1]:
# epoch마다 모델은 출력을 추출하고, 출력과 정답 사이의 오류(loss)를 게산하고, 매개변수에 대한 loss의 도함수를 수집
# 경사하강법으로 loss를 최적화(optimize)

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [4]:
# Hyperprameter는 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수.
# 서로 다른 하이퍼파라미터 값은 모델 학습과 수렴율에 영향을 미침

learning_rate = 1e-3 # 각 배치/에폭에서 모델의 매개변수를 조절하는 비율
batch_size = 64 # 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
epochs = 5 # 데이터셋을 반복하는 횟수

In [5]:
# 손실 함수(Loss fuction)는 획득한 결과와 실제 값 사이의 틀린 정도를 측정하며, 학습 중에 최소화 하려 함

# 손실 함수를 초기화
loss_fn = nn.CrossEntropyLoss()

In [6]:
# 최적화는 각 학습 단계에서 모델의 오류를 줄이기 위해 매개변수를 조절하는 과정
# 여기서는 SGD optimizer를 사용.
# Pytorch에는 ADAM, RMSProp 등 다양한 옵티마이저를 지원
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [7]:
# 학습단계에서 최적화는 3단계로 이루어짐
# optimizer.zero_grad()를 호출하여 모델 매개변수의 변화도를 재설정. 기본적으로 변화도는 더해지기(add up) 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정.
# loss.backwards()를 호출하여 예측 손실(prediction loss)을 역전파. PyTorch는 각 매개변수에 대한 손실의 변화도를 저장.
# 변화도를 계산한 뒤에는 optimizer.step()을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정.

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.309657  [    0/60000]
loss: 2.294359  [ 6400/60000]
loss: 2.277466  [12800/60000]
loss: 2.274231  [19200/60000]
loss: 2.256566  [25600/60000]
loss: 2.233011  [32000/60000]
loss: 2.236488  [38400/60000]
loss: 2.206156  [44800/60000]
loss: 2.207147  [51200/60000]
loss: 2.179990  [57600/60000]
Test Error: 
 Accuracy: 51.9%, Avg loss: 2.169818 

Epoch 2
-------------------------------
loss: 2.178375  [    0/60000]
loss: 2.169594  [ 6400/60000]
loss: 2.113901  [12800/60000]
loss: 2.129227  [19200/60000]
loss: 2.091977  [25600/60000]
loss: 2.032206  [32000/60000]
loss: 2.063269  [38400/60000]
loss: 1.985282  [44800/60000]
loss: 1.987454  [51200/60000]
loss: 1.935081  [57600/60000]
Test Error: 
 Accuracy: 59.5%, Avg loss: 1.920018 

Epoch 3
-------------------------------
loss: 1.948854  [    0/60000]
loss: 1.925099  [ 6400/60000]
loss: 1.808358  [12800/60000]
loss: 1.843313  [19200/60000]
loss: 1.759524  [25600/60000]
loss: 1.696468  [32000/600